# Quickstart

## Using the Geoscience Object API Client

Using the notebook utilities provided by `evo-objects` you can easily interact with the Geoscience Object API in a Jupyter notebook environment.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

## ObjectAPIClient

The `ObjectAPIClient` wraps endpoint functionality to provide a cohesive interface to the underlying API client implementation.

In [ ]:
from evo.objects import ObjectAPIClient

environment = manager.get_environment()
connector = manager.get_connector()

object_client = ObjectAPIClient(environment, connector, manager.cache)  # Cache is optional
service_health = await object_client.get_service_health()

print(f"Object API is {service_health.status.name.lower()}")

# The data client is an optional utility that provides helpers for uploading and downloading
# parquet data via pyarrow.Table objects
data_client = object_client.get_data_client(manager.cache)

### Listing objects

Listing a subset of objects is simple, just call the `ObjectAPIClient.list_objects()` method.

In [ ]:
offset = 0
while True:
    page = await object_client.list_objects(offset=offset, limit=10)
    if offset == 0:
        print(f"Found {page.total} object{'' if page.total == 1 else 's'}")
    for object in page:
        print(f"{object.path}: <{object.schema_id}> ({object.id})")

    if page.is_last:
        break
    else:
        offset = page.next_offset

You can also list all objects. Internally, this recursively calls the `list_objects()` method until all objects are fetched.

In [ ]:
from evo.objects import ObjectMetadata

all_objects = await object_client.list_all_objects(limit_per_request=50)


# Pretty print all objects
def display_objects(objects: list[ObjectMetadata]):
    n_objects = len(objects)
    print(f"Found {n_objects} object{'' if n_objects == 1 else 's'}")
    for object in objects:
        print(f"{object.path}: <{object.schema_id}> ({object.id})")


display_objects(all_objects)

### Create a pointset object

In [ ]:
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.csv as csv

# Import points from CSV
imported_points = csv.read_csv(
    "data/topo.csv",
    parse_options=csv.ParseOptions(delimiter=","),
    convert_options=csv.ConvertOptions(
        column_types={
            "x": pa.float64(),
            "y": pa.float64(),
            "z": pa.float64(),
        }
    ),
)

# Extract bounding box coordinates
min_x, max_x = pc.min_max(imported_points[0]).values()
min_y, max_y = pc.min_max(imported_points[1]).values()
min_z, max_z = pc.min_max(imported_points[2]).values()

sample_pointset = {
    "name": "Sample pointset",
    "uuid": None,
    "bounding_box": {
        "min_x": min_x.as_py(),
        "min_y": min_y.as_py(),
        "min_z": min_z.as_py(),
        "max_x": max_x.as_py(),
        "max_y": max_y.as_py(),
        "max_z": max_z.as_py(),
    },
    "coordinate_reference_system": {"epsg_code": 2048},
    "locations": {
        # Use the data client to save the pyarrow Table as parquet data
        "coordinates": data_client.save_table(imported_points),
    },
    "schema": "/objects/pointset/1.0.1/pointset.schema.json",
}

print(sample_pointset)

### Upload a new pointset object

In [ ]:
from evo.notebooks import FeedbackWidget

# Use the data client to upload all data referenced by the pointset
await data_client.upload_referenced_data(sample_pointset, fb=FeedbackWidget("Uploading data"))

# Use the service client to publish the pointset
new_pointset_metadata = await object_client.create_geoscience_object("sdk/v2/sample-pointset.json", sample_pointset)

# The service responds with creation metadata
print(f"{new_pointset_metadata.path}: <{new_pointset_metadata.schema_id}> ({new_pointset_metadata.id})")
print(f"\tCreated at: {new_pointset_metadata.created_at}")

### Update an existing pointset object

In [ ]:
print(f"Sample pointset has UUID: {sample_pointset['uuid']}")

updated_pointset_metadata = await object_client.update_geoscience_object(sample_pointset)

# The service responds with creation metadata
print(f"{updated_pointset_metadata.path}: <{updated_pointset_metadata.schema_id}>")
print(f"\tCreated at: {updated_pointset_metadata.created_at}")

### List versions

In [ ]:
from evo.objects import ObjectVersion

all_versions = await object_client.list_versions_by_path("sdk/v2/sample-pointset.json")


def display_versions(versions: list[ObjectVersion]) -> None:
    n_versions = len(versions)
    print(f"Found {n_versions} version{'' if n_versions == 1 else 's'}")
    for version in sorted(versions, key=lambda v: v.created_at):
        print(
            f"version:\t({version.version_id})"
            f" created on {version.created_at.date().isoformat()}"
            f" at {version.created_at.time().isoformat('seconds')}"
            f" by {version.created_by.name}"
        )


display_versions(all_versions)

### Download a pointset object

In [ ]:
from evo.notebooks import FeedbackWidget

downloaded_object = await object_client.download_object_by_path("sdk/v2/sample-pointset.json")
metadata = downloaded_object.metadata

if metadata.created_by is not None and metadata.created_by.name is not None:
    accreditation = f"{metadata.created_by.name}"
else:
    accreditation = "an unknown user"
created_at_str = metadata.created_at.astimezone().strftime("on %Y-%m-%d at %H:%M:%S")
print(f"{metadata.path} :: uploaded by {accreditation} {created_at_str}")

# Downloaded object supports JMESPath queries for more expressive access to JSON data.
print(
    downloaded_object.search(  # Project only a few fields for display
        """
        {
            name: name,
            uuid: uuid,
            schema: schema,
            coordinate_reference_system: coordinate_reference_system,
            bounding_box: {
                min: [bounding_box.min_x, bounding_box.min_y, bounding_box.min_z],
                max: [bounding_box.max_x, bounding_box.max_y, bounding_box.max_z]
            }
        }
        """
    )
)

In [ ]:
from evo.objects import DownloadedObject, ObjectReference

# If you already know the object you want, you don't even need the object client
ref = ObjectReference.new(
    environment=manager.get_environment(),
    object_path="sdk/v2/sample-pointset.json",
    version_id=metadata.version_id,  # The version ID is optional
)
print("ObjectReference URL:", ref)  # An object reference can also be a string in the format printed here.
downloaded_object = await DownloadedObject.from_reference(
    connector=manager.get_connector(),
    reference=ref,
    cache=manager.cache,
)
downloaded_object.search("@")  # Pretty-print the entire object via a JMESPath proxy object

### Download parquet data from a pointset object

In [ ]:
table_info = downloaded_object.search("locations.coordinates")  # Use a JMESPath expression to find the table info

# Download parquet data by table info reference
downloaded_data = await downloaded_object.download_table(table_info, fb=FeedbackWidget("Downloading pyarrow.Table"))

# OR you can just use the JMESPath expression in the download_table call directly
downloaded_data = await downloaded_object.download_table("locations.coordinates")
print(downloaded_data)

In [ ]:
# A similar interface can be used to download as a pandas DataFrame
await downloaded_object.download_dataframe("locations.coordinates", fb=FeedbackWidget("Downloading pandas.DataFrame"))

In [ ]:
# And as a NumPy array
await downloaded_object.download_array("locations.coordinates", fb=FeedbackWidget("Downloading numpy.ndarray"))

### Modify an object

In [ ]:
# Use the as_dict() to get the dictionary representation of the object, which can be modified
object_dict = downloaded_object.as_dict()
object_dict["description"] = "A pointset"

# Use the update() method to update the object on the Geoscience Object Service.
# By default, this prevents accidentally overwriting any updates that happened since the object was downloaded. by
# checking that the version of the downloaded object is in fact the lastest version.
new_downloaded_object = await downloaded_object.update(object_dict, check_for_conflict=True)
print(new_downloaded_object.metadata)

### Get latest object versions

In [ ]:
# Requires running the list objects from above to get the list of items
object_ids = [o.id for o in all_objects]
versions = await object_client.get_latest_object_versions(object_ids)
for uuid, version_id in versions.items():
    print(f"{uuid} -> {version_id}")

### List stages

In [ ]:
stages = await object_client.list_stages()

for stage in stages:
    print(f"{stage.name} ({stage.id})")

### Set object stage

In [ ]:
downloaded_object = await object_client.download_object_by_path("sdk/v2/sample-pointset.json")
metadata = downloaded_object.metadata
print(f"Current stage: {metadata}")

await object_client.set_stage(metadata.id, version_id=metadata.version_id, stage_id=stages[1].id)

updated_metadata = await object_client.download_object_by_id(metadata.id)
print(f"Updated stage: {updated_metadata.metadata.stage}")

## API documentation

For more information about the Geoscience Object API, visit the [Seequent developer portal](https://developer.seequent.com/docs/guides/objects).